# 📘 3주차 - 실전! 나만의 Agent Tool 만들기

### 🎯 수업 목표
1.  실용적인 문제를 해결하는 Tool을 직접 **설계**하고 **구현**할 수 있다.
2.  `requests`, `pandas`, `datetime`과 같은 외부 라이브러리를 Agent의 Tool로 연동할 수 있다.
3.  LLM이 Tool을 잘 사용하게 만드는 **좋은 `Docstring` 작성법**을 익힌다.
4.  여러 개의 Tool을 가진 Agent가 상황에 맞게 최적의 Tool을 선택하게 만들 수 있다.

### 1. 설정 및 라이브러리 설치
오늘 실습에 필요한 라이브러리들을 설치하고 API 키를 설정합니다.

In [1]:
#  기본 설정: 패키지 설치 및 API 키 설정
!pip install openai pandas requests langchain langchain_openai langchainhub -q

In [3]:
!pip install langchain-classic

In [ ]:
import openai
import os
import json
import requests
import pandas as pd
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain_classic.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.tools import tool
from langchain_classic import hub

# --- OpenAI API 키를 입력하세요 --- #
# 실제 실습에서는 dotenv이용하기!!!!!
os.environ['OPENAI_API_KEY'] = ''
# ---------------------------------- #

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

---
### 📜 Theory: 좋은 Tool 설계의 3요소

Agent에게 줄 Tool을 만들기 전, 어떤 Tool이 좋은 Tool인지 알아봅시다. LLM은 우리가 만든 함수를 코드 그 자체가 아닌, **설명(Docstring)과 함수 시그니처**를 보고 이해하기 때문에 잘 설계하는 것이 매우 중요합니다.

1.  **명확한 기능 (Single Responsibility)**
    - Tool은 복잡한 여러 기능보다, 하나의 명확한 기능만 잘 수행해야 합니다. LLM이 언제 사용해야 할지 헷갈리지 않습니다.

2.  **견고한 입출력 (Robust I/O)**
    - 함수에 들어가는 인자(Input)와 반환되는 값(Output)의 타입을 명확히 해야 합니다. (`city: str -> str`)
    - 예상되는 오류는 `try...except`로 처리하여 Agent 전체가 멈추지 않게 해야 합니다.

3.  **'마법의 주문', Docstring**
    - **Docstring은 사람을 위한 설명서가 아니라, LLM을 위한 API 명세서입니다.**
    - LLM은 Docstring을 읽고 "아, 이 Tool은 이런 상황에서, 이런 인자를 넣어서 쓰는 거구나!" 라고 판단합니다. 최대한 상세하고 명확하게 작성해야 합니다.

---
### 💻 실습 1: API 연동 Tool - 환율 조회 Agent

**🎯 목표:** `requests` 라이브러리를 사용하여 실시간 환율 정보를 가져오는 Tool을 만들고, Agent에 연결해봅니다.

**❓ 문제:** "지금 미국 달러(USD) 환율 알려줘."

In [7]:
@tool
def get_exchange_rate(currency: str) -> str:
    """지정된 통화(currency, 예: USD, EUR, JPY)의 현재 원화(KRW) 환율을 알려줍니다."""
    currency = currency.upper()
    # 안정적인 무료 환율 API 사용
    url = f"https://api.frankfurter.app/latest?from={currency}&to=KRW"
    try:
        response = requests.get(url).json()
        rate = response['rates']['KRW']
        return f"현재 1 {currency}의 환율은 {rate:.2f} KRW입니다."
    except Exception as e:
        return f"{currency} 환율 정보를 가져오는 데 실패했습니다: {e}"

# Agent 생성 및 실행
tools = [get_exchange_rate]
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "지금 미국 달러 환율 알려줘."})



> Entering new AgentExecutor chain...

Invoking: `get_exchange_rate` with `{'currency': 'USD'}`


현재 1 USD의 환율은 1474.30 KRW입니다.현재 1 미국 달러(USD)의 환율은 1,474.30 원(KRW)입니다.

> Finished chain.


{'input': '지금 미국 달러 환율 알려줘.',
 'output': '현재 1 미국 달러(USD)의 환율은 1,474.30 원(KRW)입니다.'}

---
### 💻 실습 2: 파일/Pandas 연동 Tool - CSV 요약 Agent

**🎯 목표:** `pandas` 라이브러리를 사용하여 로컬 CSV 파일의 데이터를 분석하고 요약하는 Tool을 만듭니다.

**❓ 문제:** "방금 업로드한 `sales_data.csv` 파일의 판매 데이터를 요약해줘."

In [8]:
# 실습을 위한 샘플 CSV 파일 생성
csv_data = """
Date,Product,Sales
2023-01-01,Laptop,1200
2023-01-02,Mouse,25
2023-01-03,Keyboard,75
2023-01-04,Laptop,1250
2023-01-05,Monitor,300
"""
with open("sales_data.csv", "w") as f:
    f.write(csv_data.strip())
print("'sales_data.csv' 파일이 생성되었습니다.")

'sales_data.csv' 파일이 생성되었습니다.


In [9]:
@tool
def summarize_csv(file_path: str) -> str:
    """CSV 파일의 경로를 입력받아, 데이터의 기본적인 통계 정보를 요약하여 문자열로 반환합니다."""
    try:
        df = pd.read_csv(file_path)
        return df.describe().to_string()
    except FileNotFoundError:
        return f"'{file_path}' 파일을 찾을 수 없습니다."
    except Exception as e:
        return f"파일 처리 중 오류 발생: {e}"

#  Agent 업그레이드! : 기존 Tool 리스트에 새로운 Tool을 추가합니다.
tools = [get_exchange_rate, summarize_csv]

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("--- CSV 요약 기능 테스트 ---")
agent_executor.invoke({"input": "sales_data.csv 파일의 통계를 요약해줘."})

print("\n--- 환율 조회 기능 테스트 (Tool 선택 능력 확인) ---")
agent_executor.invoke({"input": "일본 엔화(JPY) 환율 알려줘."})

--- CSV 요약 기능 테스트 ---


> Entering new AgentExecutor chain...

Invoking: `summarize_csv` with `{'file_path': 'sales_data.csv'}`


             Sales
count     5.000000
mean    570.000000
std     607.093485
min      25.000000
25%      75.000000
50%     300.000000
75%    1200.000000
max    1250.000000`sales_data.csv` 파일의 통계 요약은 다음과 같습니다:

- 데이터 개수: 5개
- 평균 매출: 570
- 표준편차: 607.09
- 최소 매출: 25
- 25% 분위 매출: 75
- 중앙값(50% 분위) 매출: 300
- 75% 분위 매출: 1200
- 최대 매출: 1250

이 데이터는 매출액에 대한 기본적인 통계 정보를 제공합니다.

> Finished chain.

--- 환율 조회 기능 테스트 (Tool 선택 능력 확인) ---


> Entering new AgentExecutor chain...

Invoking: `get_exchange_rate` with `{'currency': 'JPY'}`


현재 1 JPY의 환율은 9.43 KRW입니다.현재 1 일본 엔화(JPY)의 환율은 9.43 원(KRW)입니다.

> Finished chain.


{'input': '일본 엔화(JPY) 환율 알려줘.',
 'output': '현재 1 일본 엔화(JPY)의 환율은 9.43 원(KRW)입니다.'}

---
### 💻 실습 3: 라이브러리 활용 Tool - D-Day 계산기

**🎯 목표:** `datetime` 라이브러리를 사용하여 D-Day를 계산하는 Tool을 만들고, Agent가 자연어 질문에서 날짜를 정확히 인식하여 Tool을 호출하도록 합니다.

**❓ 문제:** "2026년 수능일인 2025년 11월 13일까지 며칠 남았어?"

In [10]:
@tool
def d_day_calculator(target_date: str) -> str:
    """목표 날짜('YYYY-MM-DD' 형식)를 기준으로 D-Day를 계산합니다. 오늘부터 목표 날짜까지 며칠 남았는지, 또는 며칠 지났는지를 알려줍니다."""
    try:
        today = datetime.now().date()
        target = datetime.strptime(target_date, "%Y-%m-%d").date()
        delta = target - today
        if delta.days == 0:
            return f"{target_date}은(는) 오늘입니다."
        elif delta.days > 0:
            return f"{target_date}까지 {delta.days}일 남았습니다."
        else:
            return f"{target_date}부터 {-delta.days}일 지났습니다."
    except ValueError:
        return "날짜 형식이 잘못되었습니다. 'YYYY-MM-DD' 형식으로 입력해주세요."
    except Exception as e:
        return f"계산 중 오류 발생: {e}"

#  최종 Agent! : 3개의 Tool을 모두 장착합니다.
tools = [get_exchange_rate, summarize_csv, d_day_calculator]

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# LLM이 자연어 질문에서 날짜를 정확히 추출하여 Tool을 호출하는지 확인!
agent_executor.invoke({"input": "2026년 수능일인 2025년 11월 13일까지 며칠 남았는지 알려줘"})




> Entering new AgentExecutor chain...

Invoking: `d_day_calculator` with `{'target_date': '2025-11-13'}`


2025-11-13부터 14일 지났습니다.2025년 11월 13일까지는 이미 14일이 지났습니다.

> Finished chain.


{'input': '2026년 수능일인 2025년 11월 13일까지 며칠 남았는지 알려줘',
 'output': '2025년 11월 13일까지는 이미 14일이 지났습니다.'}

> **성공!** LLM은 질문 속에서 날짜 정보(`2025-11-13`)를 정확히 파악하여 `d_day_calculator` Tool을 성공적으로 호출했습니다. 이제 우리 Agent는 날짜 계산 능력까지 갖추게 되었습니다.

In [11]:
# 3. Agent 실행 (Tool 선택 능력 확인)

# 3-1. 환율 질문

agent_executor.invoke({"input": "일본 엔화(JPY) 환율 알려줘."})

# 3-2. CSV 질문

agent_executor.invoke({"input": "sales_data.csv 파일의 통계를 요약해줘."})

# 3-3. D-Day 질문

agent_executor.invoke({"input": "2026년 수능일인 2025년 11월 13일까지 며칠 남았는지 알려줘"})



> Entering new AgentExecutor chain...

Invoking: `get_exchange_rate` with `{'currency': 'JPY'}`


현재 1 JPY의 환율은 9.43 KRW입니다.현재 1 일본 엔(JPY)의 환율은 9.43 원(KRW)입니다.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `summarize_csv` with `{'file_path': 'sales_data.csv'}`


             Sales
count     5.000000
mean    570.000000
std     607.093485
min      25.000000
25%      75.000000
50%     300.000000
75%    1200.000000
max    1250.000000'sales_data.csv' 파일의 통계 요약은 다음과 같습니다:

- 데이터 개수: 5개
- 평균: 570
- 표준편차: 607.09
- 최소값: 25
- 25% 분위수: 75
- 중앙값: 300
- 75% 분위수: 1200
- 최대값: 1250

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `d_day_calculator` with `{'target_date': '2025-11-13'}`


2025-11-13부터 14일 지났습니다.2025년 11월 13일까지는 이미 14일이 지났습니다. 오늘 날짜를 기준으로 계산한 결과입니다.

> Finished chain.


{'input': '2026년 수능일인 2025년 11월 13일까지 며칠 남았는지 알려줘',
 'output': '2025년 11월 13일까지는 이미 14일이 지났습니다. 오늘 날짜를 기준으로 계산한 결과입니다.'}

### 최종 테스트: 만능 Agent의 탄생
이제 우리 Agent는 환율 조회, CSV 분석, D-Day 계산이라는 세 가지의 서로 다른 능력을 모두 갖춘 **'만능 Agent'**로 거듭났습니다.

마지막 테스트는 이 Agent가 얼마나 똑똑하게 행동하는지를 보여주는 가장 중요한 과정입니다.

우리는 Agent에게 각기 다른 종류의 질문(환율, CSV, D-Day)을 던졌을 뿐이지만, Agent는 질문의 의도를 스스로 파악하고, 가지고 있는 세 개의 도구 중 현재 문제 해결에 가장 적합한 Tool을 정확히 선택하여 실행했습니다.

이것이 바로 좋은 Docstring을 가진 여러 Tool을 장착한 Agent의 강력함입니다. Agent는 더 이상 하나의 기능에 얽매이지 않고, 주어진 상황에 맞춰 능동적으로 행동하는 진정한 문제 해결사가 되었습니다.

여러분은 이제 어떤 문제든 Python 코드로 해결할 수만 있다면, 그 기능을 Agent의 일부로 만들어 자동화할 수 있는 능력을 갖추게 되었습니다.

---
### 🚀 3주차 과제 안내: 나만의 문제 해결사 Agent 만들기

**🎯 목표:** 오늘 배운 내용을 바탕으로, 자신만의 실용적인 Tool을 직접 만들고 Agent에 연결하여 문제를 해결해 봅니다.

**📌 요구사항:**
1.  **새로운 Tool 정의:** 수업에서 다룬 것 외에, 자신만의 아이디어로 새로운 Tool 함수를 1개 이상 정의하세요. (아래 아이디어 목록 참고)
2.  **라이브러리 활용:** `datetime`, `random`, `os` 등 Python 기본 라이브러리나, `BeautifulSoup4`와 같은 외부 라이브러리를 자유롭게 활용하세요.
3.  **Agent 구현:** 정의한 Tool을 LangChain Agent에 연결하고, `AgentExecutor`로 실행하세요.
4.  **시연:** 만든 Agent가 자연어 질문에 따라 Tool을 사용하여 답변하는 과정을 보여주는 코드 실행 결과를 포함하세요.

**💡 아이디어 영감 리스트:**
- **오늘 날짜/시간/요일 알려주기 Agent:** `datetime` 라이브러리 활용
- **로또 번호 생성기 Agent:** `random` 라이브러리 활용
- **현재 폴더 파일 목록 보여주기 Agent:** `os` 라이브러리 활용
- **간단한 웹사이트 제목 크롤링 Agent:** `requests`와 `BeautifulSoup4` 활용

**✅ 제출물:** 전체 실행 코드가 담긴 Jupyter Notebook (`.ipynb`) 파일